This script is to calculate the proportion of each habitat that is protected using PAD-US and equivalent Canadian datasets, broken down by owner and management intent. 

In [19]:
# Check Current Working Directory
import os
print(os.getcwd())

S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation


In [20]:
# Change working directory to the notebook's directory if needed 
# This ensures the config.py file in the same directory as the notebook can be referenced
os.chdir(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [21]:
# Module search path: Also ensures that the directory containing 'config.py' is in the Python search path ('sys.path').
import sys
sys.path.append(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [22]:
import arcpy
import os
import config

In [23]:
workspace_final = os.path.join(config.project_folder_final, f"{config.gdb_final}.gdb")
workspace_int = os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb")

In [24]:
arcpy.env.workspace = workspace_int

In [25]:
# Calculate area each habitat type intersects with the GAP 1 and 2
# Protected Areas layer using tabulate area.

in_zone_data = config.HabMap_tif
zone_field = "HabitatCod"
in_class_data = config.PA_1_2
class_field = "Loc_Ds"
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.output_int_PA)
processing_cell_size = config.HabMap_tif
classes_as_rows = "CLASSES_AS_ROWS"

arcpy.sa.TabulateArea(
    in_zone_data,
    zone_field,
    in_class_data,
    class_field,
    out_table,
    processing_cell_size,
    classes_as_rows
)

print("Table complete.")

Table complete.


In [26]:
# Sum the total count for each habitat and store values in a dictionary.

# Dictionary to store sum of count column

sum_count_PA12 = {}

# Define the category and value fields
category_field = "HabitatCod"
value_field = "Count"

with arcpy.da.SearchCursor(config.output_int_PA, [category_field, value_field]) as cursor:
    for row in cursor:
        category = row[0]
        value = row[1]
        if category in sum_count_PA12:
            sum_count_PA12[category] += value
        else:
            sum_count_PA12[category] = value

print(sum_count_PA12)


{10100: 1258962.0, 10200: 103416957.0, 10300: 7347043.0, 10302: 22378962.0, 10400: 23492058.0, 10404: 28499857.0, 10405: 15788020.0, 10500: 6790179.0, 10600: 30975.0, 10606: 1953995.0, 10607: 4035684.0, 10608: 766413.0, 10700: 899952.0, 10800: 842179.0, 10900: 2473711.0, 11000: 16842484.0, 11100: 35117377.0, 11200: 28432053.0, 11300: 1485641.0, 21400: 9454478.0, 21500: 24256013.0, 21600: 56495025.0, 21700: 10653573.0, 21800: 48600026.0, 21809: 42563748.0, 21900: 710553.0, 22100: 18275887.0, 22200: 258034.0, 32200: 6115302.0, 32300: 49404849.0, 32311: 2892313.0, 32312: 7040679.0, 32400: 9657584.0, 32413: 2658336.0, 32500: 6691529.0, 32515: 126306.0, 32516: 17223.0, 32600: 86759497.0, 32700: 20973435.0, 32800: 6339112.0, 32900: 4902208.0, 32916: 1647583.0, 33000: 1363058.0, 43610: 852868.0, 43620: 99094.0, 53810: 84053.0, 53820: 48380.0, 63917: 201064.0, 63918: 1151548.0, 64000: 7336556.0, 64100: 4669172.0, 64200: 198060.0, 64300: 4887011.0, 64400: 4039304.0, 64500: 51604854.0, 74600: 37

In [27]:
# Use Update Cursor to populate the total count values for each protected area/habitat 
# intersection in the final analysis table

# Variables
category_field = "HabitatCod"
update_field = "GAP_1_2_Count"

with arcpy.da.UpdateCursor(config.ABC_Analysis_Table, [category_field, update_field]) as cursor:
    for row in cursor:
        category = row[0]
        if category in sum_count_PA12:
            row[1] = sum_count_PA12[category]
            cursor.updateRow(row)

In [28]:
# Sum the total area for each habitat and store values in a dictionary.

# Dictionary to store sum of area column

sum_area_PA12 = {}

# Define the category and value fields
category_field = "HabitatCod"
value_field = "Area"

with arcpy.da.SearchCursor(config.output_int_PA, [category_field, value_field]) as cursor:
    for row in cursor:
        category = row[0]
        value = row[1]
        if category in sum_area_PA12:
            sum_area_PA12[category] += value
        else:
            sum_area_PA12[category] = value

print(sum_area_PA12)

{10100: 1133065800.0, 10200: 93075261300.0, 10300: 6612338700.0, 10302: 20141065800.0, 10400: 21142852200.0, 10404: 25649871300.0, 10405: 14209218000.0, 10500: 6111161100.0, 10600: 27877500.0, 10606: 1758595500.0, 10607: 3632115600.0, 10608: 689771700.0, 10700: 809956800.0, 10800: 757961100.0, 10900: 2226339900.0, 11000: 15158235600.0, 11100: 31605639300.0, 11200: 25588847700.0, 11300: 1337076900.0, 21400: 8509030200.0, 21500: 21830411700.0, 21600: 50845522500.0, 21700: 9588215700.0, 21800: 43740023400.0, 21809: 38307373200.0, 21900: 639497700.0, 22100: 16448298300.0, 22200: 232230600.0, 32200: 5503771800.0, 32300: 44464364100.0, 32311: 2603081700.0, 32312: 6336611100.0, 32400: 8691825600.0, 32413: 2392502400.0, 32500: 6022376100.0, 32515: 113675400.0, 32516: 15500700.0, 32600: 78083547300.0, 32700: 18876091500.0, 32800: 5705200800.0, 32900: 4411987200.0, 32916: 1482824700.0, 33000: 1226752200.0, 43610: 767581200.0, 43620: 89184600.0, 53810: 75647700.0, 53820: 43542000.0, 63917: 180957

In [29]:
# Use Update Cursor to populate the total area values for each protected area/habitat 
# intersection in the final analysis table

# Variables
category_field = "HabitatCod"
update_field = "GAP_1_2_Area"

with arcpy.da.UpdateCursor(config.ABC_Analysis_Table, [category_field, update_field]) as cursor:
    for row in cursor:
        category = row[0]
        if category in sum_area_PA12:
            row[1] = sum_area_PA12[category]
            cursor.updateRow(row)

In [30]:
# Use Update Cursor to calculate the proportion of each habitat that is protected
# based on the Protected_Areas_GAP_1_2 layer

with arcpy.da.UpdateCursor(os.path.join(config.project_folder_final, config.gdb_final + ".gdb", config.ABC_Analysis_Table),
                           ("Subtype_Count", "GAP_1_2_Count", "Protected_Area_1_2")) as cursor:
    for row in cursor:
        Subtype_Count = row[0]
        PAD_1_2_Count = row[1]
        
        #Ensure neither Subtype_Count nor PAD_1_2_Count is None or 0 to avoid division error
        if Subtype_Count is not None and PAD_1_2_Count is not None and Subtype_Count !=0:
            Subtype_proportion = PAD_1_2_Count/Subtype_Count
        else:
            Subtype_proportion = None 
        
        # Use an update cursor to insert new values into the table
        row[2] = Subtype_proportion
        cursor.updateRow(row)
        
print("Table updated.")

Table updated.


REPEAT STEPS FOR PROTECTED AREAS LAYER THAT INCLUDES GAP 1-3.

In [31]:
# Calculate area each habitat type intersects with the GAP 1 - 3
# Protected Areas layer using tabulate area.

in_zone_data = config.HabMap_tif
zone_field = "HabitatCod"
in_class_data = config.PA_1_2_3
class_field = "Loc_Ds"
out_table = os.path.join(config.project_folder_int, config.gdb_int + ".gdb", config.output_int_PA123)
processing_cell_size = config.HabMap_tif
classes_as_rows = "CLASSES_AS_ROWS"

arcpy.sa.TabulateArea(
    in_zone_data,
    zone_field,
    in_class_data,
    class_field,
    out_table,
    processing_cell_size,
    classes_as_rows
)

print("Table complete.")

Table complete.


In [32]:
# Sum the total count for each habitat and store values in a dictionary.

# Dictionary to store sum of count column

sum_count_PA123 = {}

# Define the category and value fields
category_field = "HabitatCod"
value_field = "Count"

with arcpy.da.SearchCursor(config.output_int_PA123, [category_field, value_field]) as cursor:
    for row in cursor:
        category = row[0]
        value = row[1]
        if category in sum_count_PA123:
            sum_count_PA123[category] += value
        else:
            sum_count_PA123[category] = value

print(sum_count_PA123)


{10100: 1990610.0, 10200: 191747881.0, 10300: 17925357.0, 10302: 62078524.0, 10400: 35282418.0, 10404: 139074045.0, 10405: 65467610.0, 10500: 52939117.0, 10600: 48472.0, 10606: 8111469.0, 10607: 15879477.0, 10608: 1591094.0, 10700: 1814431.0, 10800: 1735224.0, 10900: 4971519.0, 11000: 53678758.0, 11100: 104545191.0, 11200: 121645013.0, 11300: 4797657.0, 21400: 47449302.0, 21500: 56166286.0, 21600: 106579111.0, 21700: 21508350.0, 21800: 254016943.0, 21809: 294385328.0, 21900: 753627.0, 22100: 36352760.0, 22200: 1504012.0, 32200: 9219451.0, 32300: 122566794.0, 32311: 5906766.0, 32312: 17745263.0, 32400: 32647999.0, 32413: 8615147.0, 32500: 18232443.0, 32515: 280427.0, 32516: 18464.0, 32600: 119206683.0, 32700: 38549539.0, 32800: 30330007.0, 32900: 8390772.0, 32916: 2903298.0, 33000: 3533275.0, 43610: 2446240.0, 43620: 592691.0, 53810: 200407.0, 53820: 75927.0, 63917: 635664.0, 63918: 2094683.0, 64000: 13882497.0, 64100: 14354907.0, 64200: 478060.0, 64300: 18469296.0, 64400: 7805120.0, 64

In [33]:
# Use Update Cursor to populate the total count values for each protected area/habitat 
# intersection in the final analysis table

# Variables
category_field = "HabitatCod"
update_field = "GAP_1_2_3_Count"

with arcpy.da.UpdateCursor(config.ABC_Analysis_Table, [category_field, update_field]) as cursor:
    for row in cursor:
        category = row[0]
        if category in sum_count_PA123:
            row[1] = sum_count_PA123[category]
            cursor.updateRow(row)

In [34]:
# Sum the total area for each habitat and store values in a dictionary.

# Dictionary to store sum of area column

sum_area_PA123 = {}

# Define the category and value fields
category_field = "HabitatCod"
value_field = "Area"

with arcpy.da.SearchCursor(config.output_int_PA123, [category_field, value_field]) as cursor:
    for row in cursor:
        category = row[0]
        value = row[1]
        if category in sum_area_PA123:
            sum_area_PA123[category] += value
        else:
            sum_area_PA123[category] = value

print(sum_area_PA123)

{10100: 1791549000.0, 10200: 172573092900.0, 10300: 16132821300.0, 10302: 55870671600.0, 10400: 31754176200.0, 10404: 125166640500.0, 10405: 58920849000.0, 10500: 47645205300.0, 10600: 43624800.0, 10606: 7300322100.0, 10607: 14291529300.0, 10608: 1431984600.0, 10700: 1632987900.0, 10800: 1561701600.0, 10900: 4474367100.0, 11000: 48310882200.0, 11100: 94090671900.0, 11200: 109480511700.0, 11300: 4317891300.0, 21400: 42704371800.0, 21500: 50549657400.0, 21600: 95921199900.0, 21700: 19357515000.0, 21800: 228615248700.0, 21809: 264946795200.0, 21900: 678264300.0, 22100: 32717484000.0, 22200: 1353610800.0, 32200: 8297505900.0, 32300: 110310114600.0, 32311: 5316089400.0, 32312: 15970736700.0, 32400: 29383199100.0, 32413: 7753632300.0, 32500: 16409198700.0, 32515: 252384300.0, 32516: 16617600.0, 32600: 107286014700.0, 32700: 34694585100.0, 32800: 27297006300.0, 32900: 7551694800.0, 32916: 2612968200.0, 33000: 3179947500.0, 43610: 2201616000.0, 43620: 533421900.0, 53810: 180366300.0, 53820: 68

In [35]:
# Use Update Cursor to populate the total area values for each protected area/habitat 
# intersection in the final analysis table

# Variables
category_field = "HabitatCod"
update_field = "GAP_1_2_3_Area"

with arcpy.da.UpdateCursor(config.ABC_Analysis_Table, [category_field, update_field]) as cursor:
    for row in cursor:
        category = row[0]
        if category in sum_area_PA123:
            row[1] = sum_area_PA123[category]
            cursor.updateRow(row)

In [36]:
# Use Update Cursor to calculate the proportion of each habitat that is protected
# based on the Protected_Areas_GAP_1_2 layer

with arcpy.da.UpdateCursor(os.path.join(config.project_folder_final, config.gdb_final + ".gdb", config.ABC_Analysis_Table),
                           ("Subtype_Count", "GAP_1_2_3_Count", "Protected_Area_1_2_3")) as cursor:
    for row in cursor:
        Subtype_Count = row[0]
        PAD_1_2_3_Count = row[1]
        
        # Ensure neither Subtype_Count nor PAD_1_2_3_Count is None or 0 to avoid division error
        if Subtype_Count is not None and PAD_1_2_3_Count is not None and Subtype_Count != 0:
            Subtype_proportion = PAD_1_2_3_Count/Subtype_Count
        else:
            Subtype_proportion = None
        
        # Use an update cursor to insert new values into the table
        row[2] = Subtype_proportion
        cursor.updateRow(row)
        
print("Table updated.")

Table updated.


Calculate the proportion of protected habitat for just GAP 3 in the US.

In [37]:
# Use Update Cursor to calculate the proportion of each habitat that is protected
# based on PAD GAP 3 in the US.

with arcpy.da.UpdateCursor(os.path.join(config.project_folder_final, config.gdb_final + ".gdb", config.ABC_Analysis_Table),
                           ("Protected_Area_1_2", "Protected_Area_1_2_3", "Protected_Area_US_GAP3")) as cursor:
    for row in cursor:
        PA_1_2 = row[0]
        PA_1_2_3 = row[1]
        
        # Ensure neither PA_1_2_3 nor PA_1_2 is None or 0 to avoid division error
        if PA_1_2_3 is not None and PA_1_2 is not None and PA_1_2_3 != 0:
            GAP3_US_proportion = PA_1_2_3 - PA_1_2
        else:
            GAP3_US_proportion = None
        
        # Use an update cursor to insert new values into the table
        row[2] = GAP3_US_proportion
        cursor.updateRow(row)
        
print("Table updated.")

Table updated.
